In [1]:
import servicefoundry.core as sfy

In [2]:
sfy.login()

[yellow]You are logged in as 'cloud' with email 'cloud@truefoundry.com'[/]


In [18]:
%%writefile webapp.py

import gradio as gr
import mlfoundry as mlf
import pandas as pd
import torch

device = torch.device("cpu")

mlf.login(api_key="OGQ1NDI4ZGMtZGY3My00ZWEyLTg2NGMtZjA3OTQzNDkzZDRiOmZjNjc4Mw==", relogin=True)
client = mlf.get_client()
run = client.get_run("cloud/recommendation-system/matrix-factorization-factor20-51")
inference = run.get_model()

movies = pd.read_csv('data/movies.dat', sep='::', header=None, names=["MovieID", "Title", "Genres"], encoding='iso-8859-1')
movie_id_list = list(movies["MovieID"])
movie_names = {}
ratings = pd.read_csv('data/ratings.dat', sep='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])

for ind in movies.index:
    movie_names[movies["MovieID"][ind]]=movies["Title"][ind]

def get_recommendations(user_id: int):
    batch = []
    for movie_id in movie_id_list:
        batch.append([user_id,movie_id])
    batch = torch.tensor(batch)
    results = inference(batch[:,0].to(device), batch[:,1].to(device))
    res = list(zip(movie_id_list, results))
    res.sort(key=lambda x: x[1], reverse=True)
    return [elem[0] for elem in res[0:40]]

def get_top_recommendations(user_id: int):
    res = get_recommendations(user_id)
    id_list = set(ratings[ratings["user_id"]==user_id]["movie_id"])
    new_l = []
    i = 0
    while(len(new_l)<10):
        if res[i] not in id_list:
            new_l.append(res[i])
        i+=1
    mov_list = [movie_names[id1] for id1 in id_list]
    return [movie_names[id1] for id1 in new_l], mov_list

def greet(user_id: int):
    rec, _ = get_top_recommendations(user_id)
    return f'You top 10 recommendations are:\n\n' + f'\n'.join([f'{i+1}:   {rec[i]}' for i in range(10)])

app = gr.Interface(fn=greet, inputs="number", outputs="text")

Overwriting webapp.py


In [19]:
requirements = sfy.gather_requirements("webapp.py")
print(requirements)

{'fsspec': '2021.11.1', 'certifi': '2021.10.8', 'pytz': '2021.3', 'cryptography': '36.0.0', 'packaging': '21.3', 'unicodedata2': '13.0.0.post2', 'rich': '12.4.4', 'click': '8.1.3', 'pyarrow': '7.0.0', 'psutil': '5.9.1', 'multidict': '5.2.0', 'traitlets': '5.1.1', 'decorator': '5.1.0', 'smmap': '5.0.0', 'tqdm': '4.62.3', 'pexpect': '4.8.0', 'ujson': '4.2.0', 'gitdb': '4.0.9', 'chardet': '4.0.0', 'marshmallow': '3.17.0', 'aiohttp': '3.8.1', 'orjson': '3.7.7', 'matplotlib': '3.5.0', 'anyio': '3.4.0', 'bcrypt': '3.2.2', 'jsonschema': '3.2.0', 'idna': '3.1', 'gradio': '3.0.24', 'pyparsing': '3.0.6', 'jinja2': '3.0.3', 'threadpoolctl': '3.0.0', 'pyspark': '3.0.0', 'requests': '2.28.1', 'pycparser': '2.21', 'pygments': '2.12.0', 'paramiko': '2.11.0', 'numexpr': '2.7.3', 'markupsafe': '2.0.1', 'cloudpickle': '2.0.0', 'botocore': '1.26.10', 'urllib3': '1.26.8', 'boto3': '1.23.10', 'numpy': '1.21.2', 'six': '1.16.0', 'cffi': '1.15.0', 'puremagic': '1.14', 'torch': '1.10.0', 'backoff': '1.10.0', 

In [20]:
auto_service = sfy.Gradio("webapp.py", requirements, sfy.Parameters(
    name="demo-app",
    workspace="v1:tfy-dub-euwe1-production:recommendation-workspace",
    cpu=sfy.CPU(required=1.0),
    memory=sfy.Memory(required=1024 * 1000 * 1000)
))
auto_service.deploy()

Creating directory /home/ec2-user/SageMaker/.servicefoundry/demo-app
Created definition file /home/ec2-user/SageMaker/.servicefoundry/demo-app/servicefoundry.yaml
Created file /home/ec2-user/SageMaker/.servicefoundry/demo-app/main.py
Created file /home/ec2-user/SageMaker/.servicefoundry/demo-app/requirements.txt
Created file /home/ec2-user/SageMaker/.servicefoundry/demo-app/Procfile
Ignoring .servicefoundry
